In [ ]:
from litellm import completion
from typing import List, Dict, Any, Optional
import jinja2
import json
from abc import ABC, abstractmethod
from tavily import TavilyClient
import os


# Objective: Create a conversation object that can be used to send messages to the model and get responses and tools
class Conversation:
    def __init__(self, model: str = "gpt-4o"):  # gemini/gemini-2.0-flash
        self.model = model
        self.messages = []

    def add_message(self, role, content):
        self.messages.append({"role": role, "content": content})

    def get_response(self, tool_choice: str = "auto", tools: Optional[List] = None):
        if tools is None:
            tool_choice = None
        return completion(
            model=self.model,
            messages=self.messages,
            tools=tools,
            tool_choice=tool_choice,
        )


# Updated abstract Tool class that enforces `name` and `terminating` and accepts extra kwargs.
class Tool(ABC):
    def __init__(self, name: str, terminating: bool = False):
        self.name = name
        self.terminating = terminating

    @abstractmethod
    def run(self, conversation: Conversation) -> str:
        pass

    @abstractmethod
    def get_config(self) -> Dict[str, Any]:
        pass


# Reasoning tool (non‐terminating)
class Reasoning(Tool):
    def __init__(self, terminating: bool = False):
        super().__init__(name="reasoning", terminating=terminating)

    def run(self, conversation: Conversation) -> str:
        with open("templates/reasoning.jinja2", "r") as file:
            reasoning_template = jinja2.Template(file.read()).render()
        conversation.add_message("system", reasoning_template)
        response = conversation.get_response()
        conversation.messages.pop()  # remove the reasoning message
        return response.choices[0].message.content

    def get_config(self) -> Dict[str, Any]:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": "This tool provides reasoning for the given context.",
            },
        }


# Respond tool (terminating) – note the extra parameter `response_type`
class Respond(Tool):
    def __init__(self, response_types: List[str], terminating: bool = True):
        # Mark this tool as terminating since its response is meant to be final.
        super().__init__(name="respond", terminating=terminating)
        self.response_types = response_types

    def run(self, conversation: Conversation, response_type: str = "") -> str:
        if not response_type:
            response_type = "respond"

        with open(f"templates/{response_type}.jinja2", "r") as file:
            template_content = jinja2.Template(file.read()).render()
        conversation.add_message("system", template_content)
        response = conversation.get_response()
        conversation.messages.pop()
        return response.choices[0].message.content

    def get_config(self) -> dict:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": "This tool provides responses for the given user message.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "response_type": {
                            "type": "string",
                            "description": "The type of response to be generated.",
                            "enum": self.response_types,
                        },
                    },
                    "required": ["response_type"],
                },
            },
        }


# Search tool (non‐terminating)
class Search(Tool):
    def __init__(self, terminating: bool = False):
        super().__init__(name="search", terminating=terminating)
        self.tavily_client = TavilyClient(api_key=os.getenv("TAVILY_API_KEY"))

    def run(self, conversation: Conversation, query: str) -> str:
        response = self.tavily_client.search(query)
        return response

    def get_config(self) -> dict:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": "This tool provides search results for the given query.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The query to search for.",
                        },
                    },
                    "required": ["query"],
                },
            },
        }


# An agent is a tool that can be used to interact with the model and get responses and use tools as required.
class Agent(Tool):
    def __init__(
        self,
        name: str,
        description: str,
        fallback_tool: Tool,
        max_turns: int = 3,
        terminating: bool = False,
        tools: List[Tool] = [],
    ):
        super().__init__(name=name, terminating=terminating)
        self.description = description
        self.max_turns = max_turns
        self.available_tools = {tool.name: tool for tool in tools}
        self.fallback_tool = fallback_tool

    def run(self, conversation: Conversation) -> str:
        for turn in range(self.max_turns):
            tool, function_args = self.get_next_action(conversation)
            print(f"Using the {tool.name} tool with arguments {function_args}")
            response = tool.run(conversation, **function_args)
            if tool.terminating:
                return response

            conversation.add_message(
                "user",
                f"The {tool.name} tool has been used with args {function_args} and it responded with {response}",
            )
        return self.fallback_tool.run(conversation)

    def get_next_action(self, conversation: Conversation) -> Tool:
        response = conversation.get_response(
            tool_choice="required",
            tools=[tool.get_config() for tool in self.available_tools.values()],
        )
        tool_calls = response.choices[0].message.tool_calls
        function_name = tool_calls[0].function.name
        function_args = json.loads(tool_calls[0].function.arguments)
        return self.available_tools[function_name], function_args

    def get_config(self) -> dict:
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
            },
        }

Testing

In [24]:
with open("templates/system_message.jinja2", "r") as file:
    system_message = jinja2.Template(file.read()).render(tools="")

In [25]:
conversation = Conversation()
conversation.add_message("system", system_message)

write_report = Respond(response_types=["write_report"], terminating=True)
research_agent = Agent(
    name="research_agent",
    description="This agent is designed to help users with research.",
    tools=[Search(), Reasoning(), write_report],
    max_turns=15,
    terminating=True,
    fallback_tool=write_report,
)

respond = Respond(response_types=["respond", "clarification"])

agent = Agent(
    name="main_agent",
    description="This agent is designed to help users with market research.",
    tools=[Reasoning(), respond, research_agent],
    fallback_tool=respond,
)

conversation.add_message("user", "I want to research climate litigation")


response = agent.run(conversation)

print(response)

Using the reasoning tool with arguments {}
Using the respond tool with arguments {'response_type': 'clarification'}
Okay, I understand you want to research climate litigation. To make sure I focus on the most relevant information for you, I need to clarify a few things.

It sounds like you are interested in a broad overview of climate litigation. To confirm, are you looking to understand:

1.  The general landscape of climate litigation, including the types of cases, key players, and legal theories involved?
2.  The different jurisdictions where climate litigation is taking place (e.g., US, Europe, specific countries)?
3.  The specific types of climate litigation that are being pursued (e.g., cases against governments or fossil fuel companies)?
4.  Specific issues or legal theories related to climate litigation, such as attribution science or fiduciary duty?
5.  The intended use of your research, such as an internal briefing or academic paper?

Please let me know if this accurately ref

In [26]:
conversation.add_message("assistant", response)

conversation.add_message("user", "I want to focus on carbon capture technologies for my research. I'm interested in the general landscape, but also the emerging trends.")


response = agent.run(conversation)

print(response)

Using the reasoning tool with arguments {}
Using the research_agent tool with arguments {}
Using the search tool with arguments {'query': 'climate litigation carbon capture technologies'}
Using the reasoning tool with arguments {}
Using the search tool with arguments {'query': 'climate litigation cases carbon capture'}
Using the search tool with arguments {'query': 'climate litigation carbon capture project approvals'}
Using the reasoning tool with arguments {}
Using the search tool with arguments {'query': 'lawsuit Elk Hills carbon capture project'}
Using the reasoning tool with arguments {}

Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.


Provider List: https://docs.litellm.ai/docs/providers



RateLimitError: litellm.RateLimitError: litellm.RateLimitError: VertexAIException - {
  "error": {
    "code": 429,
    "message": "Resource has been exhausted (e.g. check quota).",
    "status": "RESOURCE_EXHAUSTED"
  }
}


In [ ]:
conversation.add_message("assistant", response)

conversation.add_message("user", "Global. No particular parties. No claims of interest, except ability to remove carbon from the atmosphere. Give me the recent developments. I want a high level overview and in-depth analysis. Please, now research.")


response = agent.run(conversation)

print(response)

Using the reasoning tool with arguments {}
Using the research_agent tool with arguments {}
Using the search tool with arguments {'query': 'recent developments climate litigation carbon capture carbon removal "carbon removal efficacy" OR "ability to remove carbon"'}
Using the search tool with arguments {'query': 'recent developments in climate litigation carbon capture "carbon removal efficacy" OR "ability to remove carbon"'}
Using the reasoning tool with arguments {}
```tool_code
print(research.search(query="climate litigation carbon capture projects legal challenges disputes regulations \"carbon removal efficacy\" OR \"carbon capture viability\" OR \"carbon capture performance\""))
```
